In [1]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print("Train samples:", x_train.shape, y_train.shape)
print("Test samples:", x_test.shape, y_test.shape)

ModuleNotFoundError: No module named 'keras'

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import necessary building blocks
from keras.models import Sequential, Model
from keras.layers import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, ZeroPadding2D, Add
from keras.layers import LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.engine.input_layer import Input
from keras.initializers import glorot_uniform

NUM_CLASSES = 10
cifar10_classes = ["airplane", "automobile", "bird", "cat", "deer",  "dog", "frog", "horse", "ship", "truck"]

# show random images from training data
cols = 8
rows = 2
fig = plt.figure(figsize=(2 * cols - 1, 2.5 * rows - 1))
for i in range(cols):
    for j in range(rows):
        random_index = np.random.randint(0, len(y_train))
        ax = fig.add_subplot(rows, cols, i * rows + j + 1)
        ax.grid('off')
        ax.axis('off')
        ax.imshow(x_train[random_index, :])
        ax.set_title(cifar10_classes[y_train[random_index, 0]])
        plt.show()

In [ ]:
X_train = x_train.astype('float32')
X_test = x_test.astype('float32')

X_train/=255
X_test/=255

Y_train = np_utils.to_categorical(y_train, len(cifar10_classes))
Y_test = np_utils.to_categorical(y_test, len(cifar10_classes))

x_val = X_train[:10000]
partial_x_train = X_train[10000:]
y_val = Y_train[:10000]
partial_y_train = Y_train[10000:]


gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
 height_shift_range=0.08, zoom_range=0.08)
val_gen = ImageDataGenerator()
train_generator = gen.flow(partial_x_train, partial_y_train, batch_size=64)
val_generator = val_gen.flow(x_val, y_val, batch_size=64)

In [ ]:
# Define the input as a tensor with shape input_shape
X_input = Input((32,32,3))

# Zero-Padding
X = ZeroPadding2D((3, 3))(X_input)

# ---------------Stage 1------------------------
X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)

# ----------Stage 2 for ResNet-------------------

# Save the input value. You'll need this later to add back to the main path.
X_shortcut = X

# Below is the script for your own ResNet:
# conv2d layer --> batch norm layer --> relu activation layer
X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv2', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
X = Activation('relu')(X)
# add extra conv2d layer chain to get equal size when combining Stage 1 and Stage 2
X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv3', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)
X = Activation('relu')(X)
# final conv2d layer and normalization
X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv4', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv4')(X)
#Add X_shortcut before doing final activation
X = Add()([X, X_shortcut])
X = Activation('relu')(X)
# output layer
X = Flatten()(X)
X = Dense(NUM_CLASSES, activation='softmax', name='fc' + str(NUM_CLASSES), kernel_initializer = glorot_uniform(seed=0))(X)

In [ ]:
# Create model
model = Model(inputs = X_input, outputs = X, name='ResNet50')
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data=val_generator)

score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

In [ ]:
aX_train = x_train.astype('float32')
X_test = x_test.astype('float32')

X_train/=255
X_test/=255

Y_train = np_utils.to_categorical(y_train, len(cifar10_classes))
Y_test = np_utils.to_categorical(y_test, len(cifar10_classes))

x_val = X_train[:10000]
partial_x_train = X_train[10000:]
y_val = Y_train[:10000]
partial_y_train = Y_train[10000:]


gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
 height_shift_range=0.08, zoom_range=0.08)
val_gen = ImageDataGenerator()
train_generator = gen.flow(partial_x_train, partial_y_train, batch_size=64)
val_generator = val_gen.flow(x_val, y_val, batch_size=64)


X_input = Input((32,32,3))
X = ZeroPadding2D(padding=(3, 3), data_format=None)(X_input)
X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
X = Activation('relu')(X)
X = MaxPooling2D((3, 3), strides=(2, 2))(X)
# Stage 2 for ResNet

# Save the input value. You'll need this later to add back to the main path.
X_shortcut = X

X = ZeroPadding2D(padding=(1, 1), data_format=None)(X)
X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv2', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv2')(X)
X = Activation('relu')(X)

X = ZeroPadding2D(padding=(1, 1), data_format=None)(X)
X = Conv2D(64, (3, 3), strides = (1, 1), name = 'conv3', kernel_initializer = glorot_uniform(seed=0))(X)
X = BatchNormalization(axis = 3, name = 'bn_conv3')(X)

#Add X_shortcut before doing final activation
X = Add()([X, X_shortcut])
X = Activation('relu')(X)

X = Flatten()(X)
X = Dense(NUM_CLASSES, activation='softmax', name='fc' + str(NUM_CLASSES), kernel_initializer = glorot_uniform(seed=0))(X)


# Create model
model = Model(inputs = X_input, outputs = X, name='ResNet50')
model.summary()


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.fit_generator(train_generator, steps_per_epoch=50000//64, epochs=10,
 validation_data=val_generator, validation_steps=10000//64)

score = model.evaluate(X_test, Y_test)
print()
print('Test accuracy: ', score[1])

In [ ]:
#Resnet architecture capable of getting 72% accuracy when tested on cifar10 data